# **Import Libraries & Data Loading**

In [ ]:
!pip install sastrawi nlp-id

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.8/54.8 MB 10.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.5/320.5 kB 12.7 MB/s eta 0:00:00
  Created wheel for nlp-id: filename=nlp_id-0.1.15.0-py3-none-any.whl size=58153892 sha256=fc68819b8ae6bba79b61a63312195f0c4acb1ae8bd013a3de7527b9e2ad77337
  Stored in directory: /root/.cache/pip/wheels/85/34/2a/deaf7b7896a1eed336b874e1a7732588d40bb7bdc0a824e73e
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9656 sha256=a8b8121c04d34c7880578331531f9a1ed089bfa35459198d308863ac6a167896
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built nlp-id wget
  Attempting uninstall: pytest
    Found existing installation: pytest 7.4.4
    Uninstalling pytest-7.4.4:
      Successfully u

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import re # if u want to learn regex [https://regex101.com/]
import string
import random # Random number generators - Library for generating random numbers, selecting random elements, shuffling sequences, etc.
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
from plotly import graph_objs as go
import plotly.express as px
import plotly.figure_factory as ff
from collections import Counter

from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator

pd.set_option("display.max_columns", None)
# pd.set_option("display.max_row", None)

In [ ]:
import nltk # Natural Language Toolkit - Library for natural language processing (NLP) tasks such as tokenization, stemming, tagging, parsing, and more.
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from tqdm import tqdm # A Fast, Extensible Progress Bar - Library for creating progress bars to monitor the progress of iterations or tasks.
from nltk.stem import WordNetLemmatizer, PorterStemmer

from nlp_id.lemmatizer import Lemmatizer
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

from tensorflow.keras.layers import Dense, Embedding, LSTM, SpatialDropout1D, Bidirectional, GlobalMaxPool1D, BatchNormalization, Dropout, GRU, Reshape
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import Tokenizer
import tensorflow as tf

import keras
from keras.initializers import Constant
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.optimizers import Adam

from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import FunctionTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    precision_score, # Precision score - Metric for evaluating classification models, measuring the ratio of correctly predicted positive observations to the total predicted positives.
    recall_score, # Recall score - Metric for evaluating classification models, measuring the ratio of correctly predicted positive observations to the total actual positives.
    f1_score, # F1 score - Harmonic mean of precision and recall, a metric for evaluating classification models.
    classification_report, # Classification report - Summary of the precision, recall, F1 score, and support for each class in a classification problem.
    accuracy_score, # Accuracy score - Metric for evaluating classification models, measuring the proportion of correct predictions to the total number of predictions.
    balanced_accuracy_score # Balanced accuracy score - Metric for evaluating classification models, measuring the accuracy of the model while accounting for imbalanced data.
)

In [ ]:
def check_duplicates(dataframe):
    print("Duplicate Values (Top 10):")
    duplicate_values = dataframe[dataframe.duplicated()]
    print(f"Number of Duplicate Rows: {duplicate_values.shape[0]}")
    display(duplicate_values.head(10))

def missing_data(data):
    total = data.isnull().sum().sort_values(ascending = False)
    percent = (data.isnull().sum()/data.isnull().count()*100).sort_values(ascending = False)
    return pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])

def basic_data_info(dataframe):
    print("Data Preview:")
    print("---------------------------")
    display(dataframe.head())

    print("\nGeneral Info:")
    print("---------------------------")
    print(dataframe.info())

    print("\nDescriptive Statistics:")
    print("---------------------------")
    display(dataframe.describe().T)

# **Load Data**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
mainPath = "/content/drive/MyDrive/Colab Notebooks/2. Satria Data 2024/Pengerjaan Satria Data"
dataPath = os.path.join(mainPath, "Dataset-BDC-SatriaData-2024")
cleanDataPath = os.path.join(mainPath, "Clean Dataset")

In [ ]:
train = pd.read_csv(dataPath + "/dataset_penyisihan_bdc_2024.csv", sep=";")
test = pd.read_csv(dataPath + "/dataset_unlabeled_penyisihan_bdc_2024.csv", sep=";")
submissions = pd.read_csv(dataPath + "/template_jawaban_penyisihan_bdc_2024.csv", sep=";")

In [ ]:
train.label.value_counts()

label
Politik                    2972
Sosial Budaya               587
Ideologi                    400
Pertahanan dan Keamanan     400
Ekonomi                     367
Sumber Daya Alam            192
Demografi                    62
Geografi                     20
Name: count, dtype: int64

# Splitting Data For Modeling

In [ ]:
train = pd.read_excel(cleanDataPath + "/balanced train.xlsx").drop(columns = "Unnamed: 0")

In [ ]:
train.label.value_counts()

label
Politik                    2969
Demografi                   427
Sosial Budaya               422
Ideologi                    343
Pertahanan dan Keamanan     331
Ekonomi                     309
Sumber Daya Alam            156
Geografi                    133
Name: count, dtype: int64

In [ ]:
# Split data into 85% train and 15% test
X_train, X_test, y_train, y_test = train_test_split(train.clean_text_5, train.label, test_size=0.15, random_state=42, stratify=train.label)

print('Train Size : ', X_train.shape)
print('Test Size  : ', X_test.shape)

Train Size :  (4326,)
Test Size  :  (764,)


In [ ]:
# Menggabungkan kembali X_train, y_train, X_test, y_test menjadi DataFrame
train_df = pd.DataFrame({'text': X_train, 'label': y_train})
test_df = pd.DataFrame({'text': X_test, 'label': y_test})

train_df = train_df.rename(columns = {"label": "label_decoded"})
test_df = test_df.rename(columns = {"label": "label_decoded"})

In [ ]:
train_df["label_decoded"].value_counts() , test_df["label_decoded"].value_counts()

(label_decoded
 Politik                    2523
 Demografi                   363
 Sosial Budaya               359
 Ideologi                    291
 Pertahanan dan Keamanan     281
 Ekonomi                     263
 Sumber Daya Alam            133
 Geografi                    113
 Name: count, dtype: int64,
 label_decoded
 Politik                    446
 Demografi                   64
 Sosial Budaya               63
 Ideologi                    52
 Pertahanan dan Keamanan     50
 Ekonomi                     46
 Sumber Daya Alam            23
 Geografi                    20
 Name: count, dtype: int64)

# Encoding Labels

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Encode label string menjadi integer
label_encoder = LabelEncoder()
train_df['label'] = label_encoder.fit_transform(train_df['label_decoded'])
test_df['label'] = label_encoder.transform(test_df['label_decoded'])

In [ ]:
# Munculkan panduan label setelah encoding
label_mapping = dict(zip(range(len(label_encoder.classes_)), label_encoder.classes_))
print("Panduan Label setelah Encoding:")
for key, value in label_mapping.items():
    print(f"Encoded {key} untuk label {value}")

Panduan Label setelah Encoding:
Encoded 0 untuk label Demografi
Encoded 1 untuk label Ekonomi
Encoded 2 untuk label Geografi
Encoded 3 untuk label Ideologi
Encoded 4 untuk label Pertahanan dan Keamanan
Encoded 5 untuk label Politik
Encoded 6 untuk label Sosial Budaya
Encoded 7 untuk label Sumber Daya Alam


# Tokenisasi dan Dataset

In [ ]:
from transformers import AutoTokenizer
from torch.utils.data import Dataset, DataLoader
import torch

# Tokenisasi
tokenizer = AutoTokenizer.from_pretrained("cahya/distilbert-base-indonesian")

class YourDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=128):
        self.texts = texts
        self.labels = labels.astype(int)  # Ensure labels are integers
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts.iloc[idx])
        label = self.labels.iloc[idx]
        encoding = self.tokenizer(text, padding='max_length', truncation=True, max_length=self.max_length, return_tensors='pt')
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

# Create datasets
train_dataset = YourDataset(train_df['text'], train_df['label'], tokenizer)
test_dataset = YourDataset(test_df['text'], test_df['label'], tokenizer)

# Define DataLoader parameters
batch_size = 16

# Create DataLoaders
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/62.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/473 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/230k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

# Distilbert-Base-Indonesian - Finetune

In [ ]:
from transformers import AutoModelForSequenceClassification, AdamW, get_scheduler
import torch
from tqdm.auto import tqdm
from sklearn.metrics import balanced_accuracy_score

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Adjust the number of labels based on the unique labels in your dataset
num_labels = len(label_encoder.classes_)

model = AutoModelForSequenceClassification.from_pretrained("cahya/distilbert-base-indonesian", num_labels=num_labels)
model.to(device)  # Pindahkan model ke GPU jika tersedia

optimizer = AdamW(model.parameters(), lr=5e-5)

num_epochs = 4
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

pytorch_model.bin:   0%|          | 0.00/273M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at cahya/distilbert-base-indonesian and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
def evaluate(model, dataloader):
    model.eval()
    total_eval_loss = 0
    total_eval_accuracy = 0
    all_preds = []
    all_labels = []
    for batch in dataloader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        with torch.no_grad():
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        logits = outputs.logits
        total_eval_loss += loss.item()

        preds = torch.argmax(logits, dim=-1)
        accuracy = (preds == labels).float().mean()
        total_eval_accuracy += accuracy.item()

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

    avg_loss = total_eval_loss / len(dataloader)
    avg_accuracy = total_eval_accuracy / len(dataloader)

    # Calculate balanced accuracy score
    balanced_acc = balanced_accuracy_score(all_labels, all_preds)

    return avg_loss, avg_accuracy, balanced_acc

In [ ]:
for epoch in range(num_epochs):
    model.train()
    total_train_loss = 0
    total_train_accuracy = 0

    for batch in tqdm(train_dataloader):
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        optimizer.zero_grad()

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        logits = outputs.logits

        total_train_loss += loss.item()

        preds = torch.argmax(logits, dim=-1)
        accuracy = (preds == labels).float().mean()
        total_train_accuracy += accuracy.item()

        loss.backward()
        optimizer.step()
        lr_scheduler.step()

    avg_train_loss = total_train_loss / len(train_dataloader)
    avg_train_accuracy = total_train_accuracy / len(train_dataloader)

    print(f"Epoch {epoch + 1}/{num_epochs}")
    print(f"Train Loss: {avg_train_loss:.4f} | Train Accuracy: {avg_train_accuracy:.4f}")

print("Training Completed")

# Evaluasi model pada test set
test_loss, test_accuracy, test_balanced_acc = evaluate(model, test_dataloader)
print(f"Test Loss: {test_loss:.4f} | Test Accuracy: {test_accuracy:.4f} | Test Balanced Accuracy: {test_balanced_acc:.4f}")

  0%|          | 0/271 [00:00<?, ?it/s]

Epoch 1/4
Train Loss: 1.0240 | Train Accuracy: 0.6766


  0%|          | 0/271 [00:00<?, ?it/s]

Epoch 2/4
Train Loss: 0.5615 | Train Accuracy: 0.8220


  0%|          | 0/271 [00:00<?, ?it/s]

Epoch 3/4
Train Loss: 0.3270 | Train Accuracy: 0.9014


  0%|          | 0/271 [00:00<?, ?it/s]

Epoch 4/4
Train Loss: 0.1759 | Train Accuracy: 0.9481
Training Completed
Test Loss: 0.7948 | Test Accuracy: 0.7687 | Test Balanced Accuracy: 0.7001


# Evaluasi Model

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report, accuracy_score, balanced_accuracy_score

# Fungsi untuk melakukan prediksi
def predict(texts, tokenizer, model, max_length=128):
    inputs = tokenizer(texts, padding=True, truncation=True, max_length=max_length, return_tensors='pt')
    input_ids = inputs['input_ids'].to(device)
    attention_mask = inputs['attention_mask'].to(device)

    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
    logits = outputs.logits
    preds = torch.argmax(logits, dim=-1)
    return preds

In [ ]:
# Melakukan prediksi pada data test
texts_to_predict = X_test.tolist()
predictions = predict(texts_to_predict, tokenizer, model)

# Decode predictions to original labels
decoded_predictions = label_encoder.inverse_transform(predictions.cpu().numpy())

In [ ]:
# Fungsi untuk evaluasi model
def evaluate_model(y_true, y_pred):
    # Precision score
    precision = precision_score(y_true, y_pred, average='weighted')

    # Recall score
    recall = recall_score(y_true, y_pred, average='weighted')

    # F1 score
    f1 = f1_score(y_true, y_pred, average='weighted')

    # Classification report
    report = classification_report(y_true, y_pred)

    # Accuracy score
    accuracy = accuracy_score(y_true, y_pred)

    # Balanced accuracy score
    balanced_accuracy = balanced_accuracy_score(y_true, y_pred)

    # Print the evaluation metrics
    print("Precision Score: ", precision)
    print("Recall Score: ", recall)
    print("F1 Score: ", f1)
    print("\nClassification Report:")
    print(report)
    print("Accuracy Score: ", accuracy)
    print("Balanced Accuracy Score: ", balanced_accuracy)

In [ ]:
# Evaluasi model dengan data test
evaluate_model(y_test, decoded_predictions)

Precision Score:  0.771403121246257
Recall Score:  0.7696335078534031
F1 Score:  0.7697371470991745

Classification Report:
                         precision    recall  f1-score   support

              Demografi       0.94      0.91      0.92        64
                Ekonomi       0.67      0.72      0.69        46
               Geografi       0.95      0.95      0.95        20
               Ideologi       0.71      0.56      0.62        52
Pertahanan dan Keamanan       0.69      0.66      0.67        50
                Politik       0.84      0.85      0.84       446
          Sosial Budaya       0.38      0.40      0.39        63
       Sumber Daya Alam       0.50      0.57      0.53        23

               accuracy                           0.77       764
              macro avg       0.71      0.70      0.70       764
           weighted avg       0.77      0.77      0.77       764

Accuracy Score:  0.7696335078534031
Balanced Accuracy Score:  0.7001137540571093


# **Saving Model**

In [ ]:
# Save the model - First Saved
modelPath = os.path.join(mainPath, "Model_Trained")

In [ ]:
model.save_pretrained(modelPath + "/Trained-Distilbert_base_indonesian_balanced_dataset")
tokenizer.save_pretrained(modelPath + "/Trained-Distilbert_base_indonesian_balanced_dataset")

('/content/drive/MyDrive/Colab Notebooks/Satria Data Another Device/Model_Trained/Trained-Distilbert_base_indonesian_balanced_dataset/tokenizer_config.json',
 '/content/drive/MyDrive/Colab Notebooks/Satria Data Another Device/Model_Trained/Trained-Distilbert_base_indonesian_balanced_dataset/special_tokens_map.json',
 '/content/drive/MyDrive/Colab Notebooks/Satria Data Another Device/Model_Trained/Trained-Distilbert_base_indonesian_balanced_dataset/vocab.txt',
 '/content/drive/MyDrive/Colab Notebooks/Satria Data Another Device/Model_Trained/Trained-Distilbert_base_indonesian_balanced_dataset/added_tokens.json',
 '/content/drive/MyDrive/Colab Notebooks/Satria Data Another Device/Model_Trained/Trained-Distilbert_base_indonesian_balanced_dataset/tokenizer.json')

In [ ]:
# # Save the model - Second Saved
# model_save_path = os.path.join(modelPath, "indobertweet_model.pth")
# torch.save(model.state_dict(), model_save_path)
# print(f"Model saved to {model_save_path}")

https://huggingface.co/docs/transformers/v4.41.3/en/tasks/multiple_choice#inference

# **Load Model**

In [ ]:
from transformers import AutoTokenizer
from torch.utils.data import Dataset, DataLoader
import torch
from transformers import AutoModelForSequenceClassification, AdamW, get_scheduler, AutoTokenizer
from tqdm.auto import tqdm
from sklearn.metrics import balanced_accuracy_score

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_path = os.path.join(modelPath, "Trained-Distilbert_base_indonesian_balanced_dataset")
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)
model.to(device)
model.eval()

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

# **Predict to Test**

In [ ]:
# Memastikan model dan tokenizer sudah dimuat sebelumnya
# model, tokenizer = load_model_and_tokenizer(model_save_path)

# Fungsi untuk melakukan prediksi
def predict(texts, tokenizer, model, max_length=128):
    inputs = tokenizer(texts, padding=True, truncation=True, max_length=max_length, return_tensors='pt')
    input_ids = inputs['input_ids'].to(device)
    attention_mask = inputs['attention_mask'].to(device)

    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)
    logits = outputs.logits
    preds = torch.argmax(logits, dim=-1)
    return preds

# Lakukan prediksi pada kolom "Text" di DataFrame
test["label"] = predict(test["Text"].tolist(), tokenizer, model).cpu().numpy()

# Decode predictions to original labels
test["label"] = label_encoder.inverse_transform(test["label"])

In [ ]:
# Menampilkan beberapa hasil prediksi
test.head()

,IDText,Text,label
0,TXT0001,Lu mau org2 pro-demokrasi di negara ini bisa p...,Politik
1,TXT0002,Prabowo ditanya soal hutang luar negeri dia me...,Politik
2,TXT0003,kiki_daliyo Ganjar Pranowo itulah beliau soso...,Ideologi
3,TXT0004,@kumparan Prabowo Gibran yang bisa melakukan i...,Politik
4,TXT0005,@sniperruben45 @uda_zulhendra @ainunnajib Lah ...,Sosial Budaya


In [ ]:
submissions["Kelas"] = test["label"]

In [ ]:
# submissions.to_csv("/content/drive/MyDrive/Colab Notebooks/2. Satria Data 2024/Pengerjaan Satria Data/Submissions/Distilbert_Base_Indonesian_Predict.csv")